In [256]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import math

import geopandas as gpd
import shapely
from shapely.geometry import Point

import re
import difflib
from collections import Counter
from collections import defaultdict

import dill
import folium


# refshape

In [257]:
df_refshape = dill.load(open('../OUTDATA/refshape_sch2015.dill','rb'))
refSHP=df_refshape['geometry'].tolist()


In [226]:
df_refshape[:3]

,name,geoid,county,centroid,centroid_long,centroid_lat,geometry,json
0,North Shore Central School District,3626370,Nassau,"(-73.6239251152, 40.8362893218)",-73.623925,40.836289,"POLYGON ((-73.67107299999999 40.859299, -73.65...","{""type"": ""FeatureCollection"", ""features"": [{""g..."
1,Seaford Union Free School District,3626400,Nassau,"(-73.4887751411, 40.5975612621)",-73.488775,40.597561,"POLYGON ((-73.50683099999999 40.666806, -73.50...","{""type"": ""FeatureCollection"", ""features"": [{""g..."
2,Uniondale Union Free School District,3629280,Nassau,"(-73.5972693079, 40.7139621285)",-73.597269,40.713962,"POLYGON ((-73.619737 40.68251799999999, -73.61...","{""type"": ""FeatureCollection"", ""features"": [{""g..."


# Pricing data from zillow

## (a) assign zillow regions to refshape

In [4]:
zillow_read=pd.read_csv('./Zillow/City_Zhvi_SingleFamilyResidence.csv')
zillow_city_ny=zillow_read[zillow_read['State'].str.contains('NY')]


In [15]:
print len(zillow_city_ny)
zillow_city_ny[:3]

956


,RegionID,RegionName,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01
0,6181,New York,NY,New York,Queens,1,NaN,NaN,NaN,NaN,...,487300,490200,492200.0,493800,495800,497900,501400,505900,510500,514000
101,13550,Rochester,NY,Rochester,Monroe,102,53100.0,54700.0,54900.0,54500.0,...,63600,63800,64000.0,64000,64000,63900,63900,63600,63600,63700
138,34937,Yonkers,NY,New York,Westchester,139,190700.0,192100.0,191100.0,189400.0,...,381700,382900,383100.0,384200,386700,388100,388500,390300,393400,395900


### (i) use zillow API to get longitude,latitude by regionid

In [ ]:
regionids_nyC=zillow_city_ny['RegionID'].tolist()


In [8]:
def z_getRegionChildren (session,regionID,df,myzillowid):

    payload = {'zws-id':myzillowid,'regionId':regionID}
    r = session.get(zillow_api_url,params=payload)
    soup=BeautifulSoup(r.text)
    main_latitude = float(soup.latitude.string)
    main_longitude = float(soup.longitude.string)
    nSubregions = int (soup.list.count.text)
    data=[]
    if nSubregions > 0:
        sub_regiontype=soup.subregiontype.string
        for sub_region in soup.list.find_all('region'):
            sub_region_id = int(sub_region.id.string)
            sub_name = sub_region.find('name').string
            sub_latitude = float(sub_region.latitude.string)
            sub_longitude = float(sub_region.longitude.string)
            if not sub_region.zindex:
                sub_price = main_price
            else: 
                sub_price = int(sub_region.zindex.string)
            data.append( (regionID,sub_regiontype,sub_region_id,sub_name,sub_latitude,sub_longitude,sub_price))
    else:
        data.append( (regionID,'main',regionID,main_name,main_latitude,main_longitude,main_price))
    return data

In [ ]:
zillow_api_url='http://www.zillow.com/webservice/GetRegionChildren.htm'
s = requests.Session()
n=0
zillow_ny_regionData = [];
for regionID in regionids_nyC: 
    main_name = zillow_nycity[zillow_nycity['RegionID']==regionID].iat[0,1]
    main_price = zillow_nycity[zillow_nycity['RegionID']==regionID].iat[0,-1]
    datatuple =  z_getRegionChildren (s,regionID,zillow_nycity,myzillowid)
    zillow_ny_regionData.extend(datatuple)
    if (n%20==0):
        print n
    n+=1
    
# save data as file
with open ('../OUTDATA/zillow_ny_regionData.csv','w') as file:
    file.write("ZIDM,ZTYPE,ZIDS,ZNAMES,ZLAT,ZLONG,ZPRICE\n")
    for line in zillow_ny_regionData:
        file.write( "%d,%s,%d,%s,%f,%f,%9.1f\n" %line)


### ii) assign zillow region/subregion to corresponding refshapes

In [258]:
# read the data
df_zillow_ny_regionData=pd.read_csv('../OUTDATA/zillow_ny_regionData.csv')
#df_zillow_ny_regionData[df_zillow_ny_regionData['ZIDM']==6181] == new york city

In [259]:
df_zillow_ny_regionData[:3]

,ZIDM,ZTYPE,ZIDS,ZNAMES,ZLAT,ZLONG,ZPRICE
0,6181,zipcode,62080,11368,40.752140,-73.853418,413100.0
1,6181,zipcode,62037,11226,40.647434,-73.958823,633400.0
2,6181,zipcode,62093,11385,40.698381,-73.887452,487800.0


In [60]:
# given a point and a list of shapes, 
#find which shape(1) the point is in
# assign to -1 if no match found

def findEncompassingShape(point,shapelist):
    for i,shape in enumerate(shapelist):
        if shape.contains(point):
            return i
    return -1

def assignPoint2Shape(sourceindex,sourcePointList,targetindex,targetShapelist):
    indexlist=[]
    for i,z in enumerate(sourcePointList): 
        p = Point(z)
        assigned_j = findEncompassingShape(p,targetShapelist)
        assigned_target_index = targetindex[assigned_j] if assigned_j > -1 else -1
        indexlist.append((sourceindex[i], assigned_target_index))
    return indexlist

# Pricing Data

In [260]:
# (1:1) assignment of each zillow region to school districts

# input data (points)
source_index = df_zillow_ny_regionData.index.values
source_points =zip(* (df_zillow_ny_regionData['ZLONG'].tolist(),df_zillow_ny_regionData['ZLAT'].tolist()))

target_index = df_refshape.index.values
target_shapes = df_refshape['geometry'].tolist()


assignment_zRegion2RefSHP = assignPoint2Shape(source_index,source_points,target_index,target_shapes)
        

In [290]:
# merge ROI data to refshape

source=df_zillow_ny_regionData
target=df_refshape

df_assign = pd.DataFrame(assignment_zRegion2RefSHP,columns=['sourceind','targetind'])
t1=pd.merge(source[['ZPRICE']],df_assign,left_index=True,right_on='sourceind')
t2=pd.merge(t1,target[['name']],left_on='targetind',right_index = True,how='inner')
print len(df_assign)
print len(t1)
print len(t2)
# select columns
#df_price_shape = t2[['RegionID','RegionName','Metro','CountyName','2006-01','2016-01','targetind']]#df_crimerate_shape[:3]
#df_price_shape.rename(columns={'targetind':'refshpindex'},inplace=True)
#df_price_shape[:3]
t2[:3]

1799
1799
1799


,ZPRICE,sourceind,targetind,name
0,413100.0,0,299,New York City Department Of Education
1,633400.0,1,299,New York City Department Of Education
2,487800.0,2,299,New York City Department Of Education


In [293]:
df1 = t2[['name','targetind']].groupby('targetind').first()

In [294]:
df2 = t2[['ZPRICE','targetind']].groupby('targetind').mean()

In [295]:
df1[:3]

,name
targetind,
0,North Shore Central School District
1,Seaford Union Free School District
2,Uniondale Union Free School District


In [296]:
df2[:3]

,ZPRICE
targetind,
0,637400.0
1,414700.0
2,411560.0


In [301]:
len(df_curprice_shape)

672

In [300]:
# combine data in the same refSHPindex using desired aggregator
t3=pd.merge(df1,df2,left_index=True,right_index=True)
df_curprice_shape=pd.merge(df_refshape[['name','geometry']],t3[['ZPRICE']],left_index=True,right_index=True,how='left')
df_curprice_shape.rename(columns={'ZPRICE':'zprice'},inplace=True)
#df_curprice_shape['index'].rename('refshpindex',inplace=True)
df_curprice_shape[:5]

,name,geometry,zprice
0,North Shore Central School District,"POLYGON ((-73.67107299999999 40.859299, -73.65...",637400.000000
1,Seaford Union Free School District,"POLYGON ((-73.50683099999999 40.666806, -73.50...",414700.000000
2,Uniondale Union Free School District,"POLYGON ((-73.619737 40.68251799999999, -73.61...",411560.000000
3,Wantagh Union Free School District,"POLYGON ((-73.52614799999999 40.691816, -73.52...",428033.333333
4,Bradford Central School District,"POLYGON ((-77.188081 42.376986, -77.182283 42....",NaN


In [302]:
dill.dump(df_curprice_shape,open('../OUTDATA/refshape_curprice.dill','wb'))

In [303]:
crs = crs = {'init': 'epsg:4326', 'no_defs': True}
geo_df = gpd.GeoDataFrame(df_curprice_shape, crs=crs)
geo_str = geo_df.to_json()

with open('../flask_actual/templates/refshape_curprice.geojson','wb') as f:
    f.write(geo_str)


# ROI Data

1. join 10yr pricing date to zillow-long-lat data (956), compute ROI
2. assign index wrt refshape
3. merge to refshape

In [14]:
print len(df_zillow_ny_regionData)
print len(assign_zRegion2RefSHP)

1799
1799


In [150]:
df_zillow_ny_regionData_bycity = df_zillow_ny_regionData[['ZLAT','ZLONG','ZIDM']].groupby('ZIDM').mean()
print len(df_zillow_ny_regionData_bycity)
df_zillow_ny_regionData_bycity[:3]

956


,ZLAT,ZLONG
ZIDM,,
3521,43.009612,-73.853321
3736,40.770760,-73.126863
3789,42.588744,-79.018007


In [156]:
# join lat-long data to pricing data
zillow_city_ny=pd.merge(zillow_city_ny,df_zillow_ny_regionData_bycity,left_on='RegionID',right_index=True,how='inner')
print len(zillow_city_ny)
zillow_city_ny[:3]


956


,RegionID,RegionName,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,ZLAT,ZLONG
0,6181,New York,NY,New York,Queens,1,NaN,NaN,NaN,NaN,...,492200.0,493800,495800,497900,501400,505900,510500,514000,40.727718,-73.940724
101,13550,Rochester,NY,Rochester,Monroe,102,53100.0,54700.0,54900.0,54500.0,...,64000.0,64000,64000,63900,63900,63600,63600,63700,43.162336,-77.609158
138,34937,Yonkers,NY,New York,Westchester,139,190700.0,192100.0,191100.0,189400.0,...,383100.0,384200,386700,388100,388500,390300,393400,395900,40.941763,-73.877682


In [157]:
zillow_city_ny.index.values[:3]

array([  0, 101, 138])

In [158]:
# (1:1) assignment of each zillow city region to refshape

# input data (points)
source_index = zillow_city_ny.index.values
source_points =zip(* (zillow_city_ny['ZLONG'].tolist(),zillow_city_ny['ZLAT'].tolist()))

target_index = df_refshape.index.values
target_shapes = df_refshape['geometry'].tolist()


assignment_zCity2RefSHP = assignPoint2Shape(source_index,source_points,target_index,target_shapes)


In [159]:
len(assignment_zCity2RefSHP)

956

In [198]:
# merge ROI data to refshape

source=zillow_city_ny
target=df_refshape

df_assign = pd.DataFrame(assignment_zCity2RefSHP,columns=['sourceind','targetind'])
t1=pd.merge(source,df_assign,left_index=True,right_on='sourceind')
t2=pd.merge(t1,target[['name']],left_on='targetind',right_index = True,how='inner')
print len(df_assign)
print len(t1)
print len(t2)
# select columns
df_price_shape = t2[['RegionID','RegionName','Metro','CountyName','2006-01','2016-01','targetind']]#df_crimerate_shape[:3]
df_price_shape.rename(columns={'targetind':'refshpindex'},inplace=True)
df_price_shape[:3]


956
956
956


,RegionID,RegionName,Metro,CountyName,2006-01,2016-01,refshpindex
0,6181,New York,New York,Queens,480300.0,514000,299
1,13550,Rochester,Rochester,Monroe,58000.0,63700,189
2,34937,Yonkers,New York,Westchester,484200.0,395900,199


In [163]:
# number of null entries at the 10 year mark
len(t2[pd.isnull(t2['2006-01'])])

29

In [199]:
# compute the N-yr ROI
duration=10 # years
df_price_shape['roi'] = (df_price_shape['2016-01']-df_price_shape['2006-01'])/df_price_shape['2006-01']/duration*100

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [200]:
# multiple zillow-regions point to the same refshape: => avg their ROI
df1 = df_price_shape[['refshpindex','roi']].groupby('refshpindex').mean()
print len(df1)
df1[:2]


509


,roi
refshpindex,
0,-0.134523
1,-1.439441


In [201]:
df2 = df_price_shape.groupby('refshpindex').first()
del df2['roi']
df2[:3]

,RegionID,RegionName,Metro,CountyName,2006-01,2016-01
refshpindex,,,,,,
0,13747,Sea Cliff,New York,Nassau,683900.0,674700
1,33904,Seaford,New York,Nassau,486300.0,416300
2,41341,Uniondale,New York,Nassau,381900.0,310500


In [202]:
df_roi = df2
df_roi['roi'] = df1['roi']

df_roi.index.names = ['refshape']
print len(df_roi)
df_roi[:3]


509


,RegionID,RegionName,Metro,CountyName,2006-01,2016-01,roi
refshape,,,,,,,
0,13747,Sea Cliff,New York,Nassau,683900.0,674700,-0.134523
1,33904,Seaford,New York,Nassau,486300.0,416300,-1.439441
2,41341,Uniondale,New York,Nassau,381900.0,310500,-1.869599


In [208]:
# to another merge with refshape to get all refshapes, including those w/o ROI data

right=df_roi
left=df_refshape

t1=pd.merge(left,right,left_index=True,right_index=True,how='left')
print len(t1)
# select columns
df_roi_shape = t1[['name','county','RegionID','Metro','roi','geometry']]
                     #df_crimerate_shape[:3]
df_roi_shape.index.names=['refshpindex']
df_roi_shape[:3]


672


,name,county,RegionID,Metro,roi,geometry
refshpindex,,,,,,
0,North Shore Central School District,Nassau,13747.0,New York,-0.134523,"POLYGON ((-73.67107299999999 40.859299, -73.65..."
1,Seaford Union Free School District,Nassau,33904.0,New York,-1.439441,"POLYGON ((-73.50683099999999 40.666806, -73.50..."
2,Uniondale Union Free School District,Nassau,41341.0,New York,-1.869599,"POLYGON ((-73.619737 40.68251799999999, -73.61..."


In [222]:
df_roi_shape[df_roi_shape['name'].str.contains('Albany')]

,name,county,RegionID,Metro,roi,geometry
refshpindex,,,,,,
210,Albany City School District,Albany,37074.0,Albany,-0.418182,"POLYGON ((-73.89421 42.709772, -73.895748 42.7..."


In [211]:
dill.dump(df_roi_shape,open('../OUTDATA/refshape_roi_2006-2016.dill','wb'))

In [212]:
crs = crs = {'init': 'epsg:4326', 'no_defs': True}
geo_df = gpd.GeoDataFrame(df_roi_shape, crs=crs)
geo_str = geo_df.to_json()

In [213]:
print geo_df['roi'].describe()

count    498.000000
mean       0.164713
std        1.959464
min       -4.122191
25%       -1.472757
50%        0.240421
75%        1.481885
max        5.689888
Name: roi, dtype: float64


In [227]:
# function to provide color (from colorbrewer)
def getColor(d):
    if d > 5: 
        return '#313695' # blue
    elif d > 4:
        return '#4575b4'
    elif d > 3:
        return '#74add1'
    elif d > 2:
        return '#abd9e9'
    elif d > 1:
        return '#e0f3f8'
    elif d > 0:
        return '#fee090'
    elif d > -1:
        return '#fdae61'
    elif d > -2:
        return '#f46d43'
    elif d > -3:    
        return '#d73027'
    elif d > -4: 
        return '#a50026' # red
    return '#d9d9d9'


In [238]:
#
m = folium.Map([42.75,-73.938], zoom_start=7,tiles='cartodbpositron')

folium.GeoJson(geo_df, style_function=lambda feature: {
        'fillColor': getColor(feature['properties']['roi']),
        'color' : 'black',
        'weight' : 1,
        'fillOpacity':1, 'opacity':0.2
        }
              ).add_to(m)
path='roi_map.html'
m.save(path)


In [252]:
# use colormap so we can include legend

from folium.colormap import LinearColormap
vmin=math.floor(geo_df['roi'].min())
vmax=math.ceil(geo_df['roi'].max())
colormap = LinearColormap(['#313695','#fee090','#a50026'], vmin=vmin,vmax=vmax) # red,yellow,blue

m = folium.Map([42.75,-73.938], zoom_start=7,tiles='cartodbpositron')

folium.GeoJson(geo_df, style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['roi']),
        'color' : 'black',
        'weight' : 1,
        'fillOpacity':1, 'opacity':0.2
        }
              ).add_to(m)
colormap.caption = 'ROI color scale'
colormap.add_to(m)

path='roi_map.html'
m.save(path)


In [248]:
colormap

In [237]:
#
m = folium.Map([42.75,-73.938], zoom_start=7,tiles='cartodbpositron')
path='test.html'
m.save(path)


In [255]:
with open('../OUTDATA/refshape_roi_2006-2016.json','wb') as f:
    f.write(geo_str)